In [1]:
import numpy as np
import sympy as sp
from scipy.optimize import fsolve
from scipy.integrate import odeint

In [2]:
def tf(y):
    y1, y2 = y
    return np.array([-y1**2, y2**(1./2.)])
def tdf(y):
    y1, y2 = y
    return np.array([[-2.*y1, 0], [0, 0.5*y2**(-0.5)]])
def K(x, y, sigma = 1):
    return np.exp(-1*(np.linalg.norm(x - y)**2)/(sigma**2))

# Unidimensional system
def f(y, t, sigma = 1):
    y1, y2, y3, a1, a2, a3 = y
    return np.array([K(y1, y1, sigma)*a1 + K(y1, y2, sigma)*a2 + K(y1, y3, sigma)*a3,
                    K(y2, y1, sigma)*a1 + K(y2, y2, sigma)*a2 + K(y2, y3, sigma)*a3,
                    K(y3, y1, sigma)*a1 + K(y3, y2, sigma)*a2 + K(y3, y3, sigma)*a3,
                    -2./sigma*(np.dot(a1.T, a1)*K(y1, y1, sigma)*(y1 - y1) + np.dot(a1.T, a2)*K(y1, y2, sigma)*(y1 - y2) + np.dot(a1.T, a3)*K(y1, y3, sigma)*(y1 - y3)),
                    -2./sigma*(np.dot(a2.T, a1)*K(y2, y1, sigma)*(y2 - y1) + np.dot(a2.T, a2)*K(y2, y2, sigma)*(y2 - y2) + np.dot(a2.T, a3)*K(y2, y3, sigma)*(y2 - y3)),
                    -2./sigma*(np.dot(a3.T, a1)*K(y3, y1, sigma)*(y3 - y1) + np.dot(a3.T, a2)*K(y3, y2, sigma)*(y3 - y2) + np.dot(a3.T, a3)*K(y3, y3, sigma)*(y3 - y3))])

def df(y, t, sigma = 1):
    y1, y2, y3, a1, a2, a3 = y
    return np.array([[(2./sigma)*y1*(K(y1, y1, sigma)*a1 + K(y1, y2, sigma)*a2 + K(y1, y3, sigma)*a3),
                      (-2./sigma)*y2*K(y1, y2, sigma)*a2,
                      (-2./sigma)*y3*K(y1, y3, sigma)*a3,
                      K(y1, y1, sigma), K(y1, y2, sigma), K(y1, y3, sigma)],
                     [(-2./sigma)*y1*K(y2, y1, sigma)*a1,
                      (2./sigma)*y2*(K(y2, y1, sigma)*a1 + K(y2, y2, sigma)*a2 + K(y2, y3, sigma)*a3),
                      (-2./sigma)*y3*K(y2, y3, sigma)*a3,
                      K(y2, y1, sigma), K(y2, y2, sigma), K(y2, y3, sigma)],
                     [(-2./sigma)*y1*K(y1, y1, sigma)*a1,
                      (-2./sigma)*y2*K(y1, y2, sigma)*a2,
                      (2./sigma)*y3*(K(y1, y1, sigma)*a1 + K(y1, y2, sigma)*a2 + K(y1, y3, sigma)*a3),
                      K(y3, y1, sigma), K(y3, y2, sigma), K(y3, y3, sigma)],
                     [-2./sigma*(-1*np.dot(a1.T, a2)*K(y1, y2, sigma)*(y1 - y2) + np.dot(a1.T, a3)*K(y1, y3, sigma)*(y1 - y3))],
                     [],
                     []])

## Function of the Differential Problem
#def f(y):
#   y1, y2 = y
#   return np.array([-y1 + y2, 3*y1 - 7*y2])

## Jacobian of the problem
#def df(y):
#    y1, y2 = y
#    return np.array([[-1., 1.], [3., -7.]])

## Real (analytic) solution of differential problem
def sol(y):
    return np.array([1./(1. + y), ((1./4.)*(y + 2)**2)])

## F for solve F(y) = 0 with Newton method. 
## y = y_i+1
def F(y, y_0, h, f):
    return y - y_0 - h*f(y)

## Jacobian of F
def dF(y, h, df):
    return np.identity(2) - h*df(y)

## Implicit midpoint Rule
## Here we use y_i+1 = y_1 + h*f((y_i+1 + y_i)/2)
def Fi(y, y_0, h, f):
    return y - y_0 - h*f((y+y_0)*0.5)

## Jacobian of Fi, using chain's rule.
def dFi(y, h, df):
    return np.identity(2) - 0.5*h*df(y)

def newton(y_0, h, f, df, maxiter = 10000, tol = 1e-10):
    #Initial Guess
    y = y_0.copy()
    for j in range(maxiter):
        #Step calculation
        w = np.linalg.solve(dFi(y, h, df), -1*Fi(y, y_0, h, f))
        y += w
        ## Found an acceptable solution
        if np.linalg.norm(F(y, y_0, h, f)) < tol:
            break
    return y

def solver(f, df, h, y_0, t_0, t_f):
    n = int((t_f-t_0)/h)
    y = np.zeros((n+1,len(y_0)))
    trange = np.linspace(t_0, t_f, n+1)
    y[0] = y_0.copy()
    for i in range(1,n+1):
        y[i] = newton(y[i-1], h, f, df)
    return y, trange

In [36]:
# Shooting Method Version 1
# dy: vectorized function
# dz: vectorized function
# a: left border
# b: right border
# y_a: vector value in a
# y_b: vector value in b
# alpha: vector shooting
# h: dt
def shootingMethod1(dy, dz, a, b, y_a, y_b, alpha, h):
    y_i = y_a
    z_i = alpha
    for i in np.arange(a, b, h):
        y_i_next = y_i + dy(i, y_i, z_i) * h
        z_i_next = z_i + dz(i, y_i, z_i) * h
        y_i = y_i_next
        z_i = z_i_next
    print "y_b real:", y_b
    print "con alpha =", alpha, "se obtiene:", y_i
    print "diferencia:", y_b - y_i

def dy1(i, y_i, z_i):
    return z_i

# Test function: d^2 y/dx^2 = (3/2)y^2
# Fuente: wikipedia
# Aproximacion a Soluciones: -8 y -36
def dz1(i, y, z):
    return 1.5 * (y ** 2)

# Test function: d^2 y/dx^2 = -8x^2 + 72x + 2y
# Fuente: youtube
# Aproximacion a Soluciones: -20.57
def dz2(i, y, z):
    return -8 * (i ** 2) + 72 * i + 2 * y

# -------------------------------------------------------------------------

In [38]:
#vdy1 = np.vectorize(dy1)
#vdz1 = np.vectorize(dz1)
#shootingMethod1(vdy1, vdz1, 0, 1, np.array([1,2,3,4]), np.array([2,3,4,5]), np.array([1,1,1,1]), 0.01)
shootingMethod1(dy1, dz1, 0, 1, 4, 1, -36, 0.00001)
shootingMethod1(dy1, dz1, 0, 1, 4, 1, -8, 0.00001)
shootingMethod1(dy1, dz2, 0, 9, 0, 0, -20.57, 3)

y_b real: 1
con alpha = -36 se obtiene: 1.06240553676
diferencia: -0.0624055367579
y_b real: 1
con alpha = -8 se obtiene: 1.00002699429
diferencia: -2.69942932307e-05
y_b real: 0
con alpha = -20.57 se obtiene: 0.09
diferencia: -0.09


In [3]:
y, t = solver(tf, tdf, 0.1, np.array([1,1]), 0, 1)
y

array([[ 1.        ,  1.        ],
       [ 0.9089023 ,  1.10253125],
       [ 0.83304297,  1.21006249],
       [ 0.76888837,  1.32259374],
       [ 0.71392033,  1.44012498],
       [ 0.66629544,  1.56265623],
       [ 0.62463303,  1.69018747],
       [ 0.58787843,  1.82271872],
       [ 0.55521203,  1.96024996],
       [ 0.52598724,  2.10278121],
       [ 0.49968704,  2.25031245]])

In [4]:
sol(t).T

array([[ 1.        ,  1.        ],
       [ 0.90909091,  1.1025    ],
       [ 0.83333333,  1.21      ],
       [ 0.76923077,  1.3225    ],
       [ 0.71428571,  1.44      ],
       [ 0.66666667,  1.5625    ],
       [ 0.625     ,  1.69      ],
       [ 0.58823529,  1.8225    ],
       [ 0.55555556,  1.96      ],
       [ 0.52631579,  2.1025    ],
       [ 0.5       ,  2.25      ]])

In [5]:
y

array([[ 1.        ,  1.        ],
       [ 0.9089023 ,  1.10253125],
       [ 0.83304297,  1.21006249],
       [ 0.76888837,  1.32259374],
       [ 0.71392033,  1.44012498],
       [ 0.66629544,  1.56265623],
       [ 0.62463303,  1.69018747],
       [ 0.58787843,  1.82271872],
       [ 0.55521203,  1.96024996],
       [ 0.52598724,  2.10278121],
       [ 0.49968704,  2.25031245]])

In [11]:
t0 = np.array([0,10,-1,1,-2,3])
print odeint(f, t0, t)

[[  0.          10.          -1.           1.          -2.           3.        ]
 [  0.21894485   9.8         -0.6521391    0.78497925  -2.           3.21502075]
 [  0.46045238   9.6         -0.28520285   0.5943511   -2.           3.4056489 ]
 [  0.72903461   9.4          0.09526225   0.44113442  -2.           3.55886558]
 [  1.02407133   9.2          0.48413312   0.32672904  -2.           3.67327096]
 [  1.34168134   9.           0.87780804   0.24467205  -2.           3.75532795]
 [  1.67710794   8.8          1.27413682   0.18653704  -2.           3.81346296]
 [  2.02608763   8.6          1.67193456   0.14512496  -2.           3.85487504]
 [  2.38526015   8.4          2.07056388   0.11518795  -2.           3.88481205]
 [  2.75211921   8.2          2.4696792    0.09314054  -2.           3.90685946]
 [  3.12483199   8.           2.86908857   0.0765845   -2.           3.9234155 ]]


In [12]:
0.0765845 +  -2.  +         3.9234155

2.0